# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
#%pip install -r requirements.txt

In [2]:
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import math
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
# from imblearn.under_sampling import RandomUnderSampler


# if 'PG_PORT' in os.environ:

#     host = 'db'
#     port = os.environ['PG_PORT']
    # database = os.environ['PG_DATABASE']
#     user = os.environ['PG_USER']
#     password = os.environ['PG_PASSWORD']
    
#     # pd.read_sql用のコネクタ
#     conn = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

#     df_customer = pd.read_sql(sql='select * from customer', con=conn)
#     df_category = pd.read_sql(sql='select * from category', con=conn)
#     df_product = pd.read_sql(sql='select * from product', con=conn)
#     df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
#     df_store = pd.read_sql(sql='select * from store', con=conn)
#     df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

# else:
#     if not os.path.exists('../data/'):
#         !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
#         os.chdir('100knocks-preprocess/docker/work/answer')

dtype = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str
}

df_customer = pd.read_csv("../data/customer.csv", dtype=dtype)
df_category = pd.read_csv("../data/category.csv", dtype=dtype)
df_product = pd.read_csv("../data/product.csv", dtype=dtype)
df_receipt = pd.read_csv("../data/receipt.csv", dtype=dtype)
df_store = pd.read_csv("../data/store.csv", dtype=dtype)
df_geocode = pd.read_csv("../data/geocode.csv", dtype=dtype)


# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [92]:
df_receipt.head(10)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
1202    20190903   1567468800   S14011        1192               2   
62317   20190304   1551657600   S14046        1152               2   
54095   20170921   1505952000   S14028        1192               2   
72747   20190724   1563926400   S13009        1192               2   
4632    20170721   1500595200   S13015        1112               1   
10320   20190103   1546473600   S14021        1162               2   
28304   20180621   1529539200   S13004        1172               2   
97294   20190209   1549670400   S14021        1142               2   
20754   20180108   1515369600   S13001        1122               1   
21847   20190830   1567123200   S13035        1182               1   

          customer_id  product_cd  quantity  amount  rank  
1202   CS011415000006  P080104055         1   10925     1  
62317  ZZ000000000000  P090302118         1    6800     2  
54095  CS028605000002  P090907210         1    5780     3  
72747  ZZ000000000000  P090907209         1    5480     4  
4632   CS015515000034  P090907208         1    5480     5  
10320  ZZ000000000000  P090907208         1    5480     6  
28304  ZZ000000000000  P090907206         1    5440     7  
97294  CS021515000089  P090907206         1    5440     8  
20754  ZZ000000000000  P090901100         1    5280     9  
21847  CS035615000040  P071301009         1    5280    10

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [93]:
colum = ['sales_ymd', 'customer_id', 'product_cd', 'amount']
df_receipt[colum].head(10)

sales_ymd     customer_id  product_cd  amount
1202    20190903  CS011415000006  P080104055   10925
62317   20190304  ZZ000000000000  P090302118    6800
54095   20170921  CS028605000002  P090907210    5780
72747   20190724  ZZ000000000000  P090907209    5480
4632    20170721  CS015515000034  P090907208    5480
10320   20190103  ZZ000000000000  P090907208    5480
28304   20180621  ZZ000000000000  P090907206    5440
97294   20190209  CS021515000089  P090907206    5440
20754   20180108  ZZ000000000000  P090901100    5280
21847   20190830  CS035615000040  P071301009    5280

---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [94]:

colum = df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

colum = df_receipt.rename(columns = {'sales_ymd':'sales_data'})

colum.head(10)

sales_data  sales_epoch store_cd  receipt_no  receipt_sub_no  \
1202     20190903   1567468800   S14011        1192               2   
62317    20190304   1551657600   S14046        1152               2   
54095    20170921   1505952000   S14028        1192               2   
72747    20190724   1563926400   S13009        1192               2   
4632     20170721   1500595200   S13015        1112               1   
10320    20190103   1546473600   S14021        1162               2   
28304    20180621   1529539200   S13004        1172               2   
97294    20190209   1549670400   S14021        1142               2   
20754    20180108   1515369600   S13001        1122               1   
21847    20190830   1567123200   S13035        1182               1   

          customer_id  product_cd  quantity  amount  rank  
1202   CS011415000006  P080104055         1   10925     1  
62317  ZZ000000000000  P090302118         1    6800     2  
54095  CS028605000002  P090907210         1    5780     3  
72747  ZZ000000000000  P090907209         1    5480     4  
4632   CS015515000034  P090907208         1    5480     5  
10320  ZZ000000000000  P090907208         1    5480     6  
28304  ZZ000000000000  P090907206         1    5440     7  
97294  CS021515000089  P090907206         1    5440     8  
20754  ZZ000000000000  P090901100         1    5280     9  
21847  CS035615000040  P071301009         1    5280    10

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [95]:
filter_id = df_receipt[df_receipt['customer_id'] == 'CS018205000001']
df_filter_id = filter_id[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(df_filter_id)

       sales_ymd     customer_id  product_cd  amount
68117   20190226  CS018205000001  P071401020    2200
36      20180911  CS018205000001  P071401012    2200
72254   20180911  CS018205000001  P071401005    1100
21110   20170614  CS018205000001  P050206001     990
9843    20180414  CS018205000001  P060104007     600
58121   20190924  CS018205000001  P060805001     495
91525   20190924  CS018205000001  P091503001     280
28757   20180414  CS018205000001  P071101002     278
88508   20190216  CS018205000001  P040101002     218
39256   20190226  CS018205000001  P070902035     168
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [96]:
filter_id_amo = df_receipt[(df_receipt['customer_id'] == 'CS018205000001') &
                           (df_receipt['amount'] >= 1000)]
df_filter = filter_id_amo[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(df_filter)

       sales_ymd     customer_id  product_cd  amount
68117   20190226  CS018205000001  P071401020    2200
36      20180911  CS018205000001  P071401012    2200
72254   20180911  CS018205000001  P071401005    1100


---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
filter_id_amo_or_qua = df_receipt[(df_receipt['customer_id'] == 'CS018205000001') &
                           ((df_receipt['amount'] >= 1000) | (df_receipt['quantity']>= 5))]
df_filter = filter_id_amo_or_qua[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(df_filter)

       sales_ymd     customer_id  product_cd  amount
68117   20190226  CS018205000001  P071401020    2200
36      20180911  CS018205000001  P071401012    2200
72254   20180911  CS018205000001  P071401005    1100
9843    20180414  CS018205000001  P060104007     600


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [98]:
filter_id_amo2 = df_receipt[(df_receipt['customer_id'] == 'CS018205000001') &
                           (df_receipt['amount'].between(1000,2000))]
df_filter = filter_id_amo2[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(df_filter)

       sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [99]:
id_cd = df_receipt[(df_receipt['customer_id'] == 'CS018205000001') &
                           (df_receipt['product_cd'] != 'P071401019' )]
df_filter = id_cd[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
print(df_filter)

       sales_ymd     customer_id  product_cd  amount
68117   20190226  CS018205000001  P071401020    2200
36      20180911  CS018205000001  P071401012    2200
72254   20180911  CS018205000001  P071401005    1100
21110   20170614  CS018205000001  P050206001     990
9843    20180414  CS018205000001  P060104007     600
58121   20190924  CS018205000001  P060805001     495
91525   20190924  CS018205000001  P091503001     280
28757   20180414  CS018205000001  P071101002     278
88508   20190216  CS018205000001  P040101002     218
39256   20190226  CS018205000001  P070902035     168
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [100]:
df_store.query('prefecture_cd !="13" and floor_area <= 900')

store_cd store_name prefecture_cd prefecture            address  \
18   S14046       北山田店            14       神奈川県   神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店            14       神奈川県  神奈川県横浜市港北区日吉本町四丁目   
38   S12013       習志野店            12        千葉県       千葉県習志野市芝園一丁目   

                      address_kana        tel_no  longitude  latitude  \
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
38             チバケンナラシノシシバゾノイッチョウメ  047-123-4002   140.0220  35.66122   

    floor_area  
18       831.0  
20       890.0  
38       808.0

In [101]:
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')

store_cd store_name prefecture_cd prefecture            address  \
18   S14046       北山田店            14       神奈川県   神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店            14       神奈川県  神奈川県横浜市港北区日吉本町四丁目   
38   S12013       習志野店            12        千葉県       千葉県習志野市芝園一丁目   

                      address_kana        tel_no  longitude  latitude  \
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
38             チバケンナラシノシシバゾノイッチョウメ  047-123-4002   140.0220  35.66122   

    floor_area  
18       831.0  
20       890.0  
38       808.0

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [102]:
cd = df_store[df_store['store_cd'].str.startswith('S14')]
cd

store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
22   S14034      川崎野川店            14       神奈川県         神奈川県川崎市宮前区野川   
26   S14048      中川中央店            14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
28   S14026     辻堂西海岸店            14       神奈川県      神奈川県藤沢市辻堂西海岸二丁目   
31   S14049      川崎大師店            14       神奈川県      神奈川県川崎市川崎区中瀬三丁目   
32   S14023        川崎店            14       神奈川県      神奈川県川崎市川崎区本町二丁目   
35   S14027       南藤沢店            14       神奈川県           神奈川県藤沢市南藤沢   
36   S14021       伊勢原店            14       神奈川県       神奈川県伊勢原市伊勢原四丁目   
37   S14047       相模原店            14       神奈川県       神奈川県相模原市千代田六丁目   
40   S14042       新山下店            14       神奈川県      神奈川県横浜市中区新山下二丁目   
44   S14025        大和店            14       神奈川県           神奈川県大和市下和田   
45   S14045        厚木店            14       神奈川県         神奈川県厚木市中町二丁目   
51   S14024        三田店            14       神奈川県      神奈川県川崎市多摩区三田四丁目   
52   S14006       葛が谷店            14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
22             カナガワケンカワサキシミヤマエクノガワ  044-123-4044   139.5998  35.57693   
26   カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ  045-123-4051   139.5758  35.54912   
28      カナガワケンフジサワシツジドウニシカイガンニチョウメ  046-123-4040   139.4466  35.32464   
31       カナガワケンカワサキシカワサキクナカゼサンチョウメ  044-123-4052   139.7327  35.53759   
32      カナガワケンカワサキシカワサキクホンチョウニチョウメ  044-123-4037   139.7028  35.53599   
35              カナガワケンフジサワシミナミフジサワ  046-123-4041   139.4896  35.33762   
36           カナガワケンイセハラシイセハラヨンチョウメ  046-123-4035   139.3129  35.40169   
37           カナガワケンサガミハラシチヨダロクチョウメ  042-123-4050   139.3748  35.55959   
40       カナガワケンヨコハマシナカクシンヤマシタニチョウメ  045-123-4047   139.6593  35.43894   
44                  カナガワケンヤマトシシモワダ  046-123-4039   139.4680  35.43414   
45            カナガワケンアツギシナカチョウニチョウメ  046-123-4048   139.3651  35.44182   
51          カナガワケンカワサキシタマクミタヨンチョウメ  044-123-4038   139.5424  35.60770   
52             カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633  35.53573   

    floor_area  
2       1732.0  
3       1495.0  
4       1679.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
19      1838.0  
20       890.0  
22      1318.0  
26      1657.0  
28      1732.0  
31       962.0  
32      1804.0  
35      1521.0  
36       962.0  
37      1047.0  
40      1044.0  
44      1011.0  
45       980.0  
51       972.0  
52      1886.0

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [103]:
id = df_customer[df_customer['customer_id'].str.endswith('1')]
id.head(10)

customer_id customer_name gender_cd gender  birth_day  age postal_cd  \
1    CS037613000071         六角 雅彦         9     不明 1952-04-01   66  136-0076   
3    CS028811000001        堀井 かおり         1     女性 1933-03-27   86  245-0016   
14   CS040412000191         川井 郁恵         1     女性 1977-01-05   42  226-0021   
31   CS028314000011        小菅 あおい         1     女性 1983-11-26   35  246-0038   
56   CS039212000051        藤島 恵梨香         1     女性 1997-02-03   22  166-0001   
59   CS015412000111         松居 奈月         1     女性 1972-10-04   46  136-0071   
63   CS004702000041          野島 洋         0     男性 1943-08-24   75  176-0022   
74   CS041515000001         栗田 千夏         1     女性 1967-01-02   52  206-0001   
85   CS029313000221        北条 ひかり         1     女性 1987-06-19   31  279-0011   
102  CS034312000071         望月 奈央         1     女性 1980-09-20   38  213-0026   

                     address application_store_cd  application_date  \
1         東京都江東区南砂**********               S13037          20150414   
3     神奈川県横浜市泉区和泉町**********               S14028          20160115   
14   神奈川県横浜市緑区北八朔町**********               S14040          20151101   
31    神奈川県横浜市瀬谷区宮沢**********               S14028          20151123   
56      東京都杉並区阿佐谷北**********               S13039          20171121   
59        東京都江東区亀戸**********               S13015          20150629   
63        東京都練馬区向山**********               S13004          20170218   
74        東京都多摩市和田**********               S13041          20160422   
85        千葉県浦安市美浜**********               S12029          20180810   
102   神奈川県川崎市高津区久末**********               S14034          20160106   

        status_cd  
1    0-00000000-0  
3    0-00000000-0  
14   1-20091025-4  
31   1-20080426-5  
56   1-20100215-4  
59   0-00000000-0  
63   0-00000000-0  
74   E-20100803-F  
85   0-00000000-0  
102  0-00000000-0

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [104]:
store = df_store[df_store['address'].str.contains('横浜市')]
store

store_cd store_name prefecture_cd prefecture              address  \
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
26   S14048      中川中央店            14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
40   S14042       新山下店            14       神奈川県      神奈川県横浜市中区新山下二丁目   
52   S14006       葛が谷店            14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
26   カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ  045-123-4051   139.5758  35.54912   
40       カナガワケンヨコハマシナカクシンヤマシタニチョウメ  045-123-4047   139.6593  35.43894   
52             カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633  35.53573   

    floor_area  
2       1732.0  
3       1495.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
20       890.0  
26      1657.0  
40      1044.0  
52      1886.0

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [105]:
status = df_customer[df_customer['status_cd'].str.match('^[A-F]')]
status.head(10)

customer_id customer_name gender_cd gender  birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子         1     女性 1976-10-04   42  151-0053   
6   CS015414000103         奥野 陽子         1     女性 1977-08-09   41  136-0073   
12  CS011215000048         芦田 沙耶         1     女性 1992-02-01   27  223-0062   
15  CS029415000023         梅田 里穂         1     女性 1976-01-17   43  279-0043   
21  CS035415000029         寺沢 真希         9     不明 1977-09-27   41  158-0096   
32  CS031415000106        宇野 由美子         1     女性 1970-02-26   49  151-0053   
33  CS029215000025         石倉 美帆         1     女性 1993-09-28   25  279-0022   
40  CS033605000005         猪股 雄太         0     男性 1955-12-05   63  246-0031   
44  CS033415000229        板垣 菜々美         1     女性 1977-11-07   41  246-0021   
53  CS008415000145         黒谷 麻緒         1     女性 1977-06-27   41  157-0067   

                     address application_store_cd  application_date  \
2        東京都渋谷区代々木**********               S13031          20150529   
6         東京都江東区北砂**********               S13015          20150722   
12  神奈川県横浜市港北区日吉本町**********               S14011          20150228   
15       千葉県浦安市富士見**********               S12029          20150610   
21      東京都世田谷区玉川台**********               S13035          20141220   
32       東京都渋谷区代々木**********               S13031          20150201   
33        千葉県浦安市今川**********               S12029          20150708   
40    神奈川県横浜市瀬谷区瀬谷**********               S14033          20150425   
44  神奈川県横浜市瀬谷区二ツ橋町**********               S14033          20150712   
53      東京都世田谷区喜多見**********               S13008          20150829   

       status_cd  
2   D-20100325-C  
6   B-20100609-B  
12  C-20100421-9  
15  D-20100918-E  
21  F-20101029-F  
32  F-20100511-E  
33  B-20100820-C  
40  F-20100917-E  
44  F-20100326-E  
53  F-20100622-F

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
status = df_customer[df_customer['status_cd'].str.
endswith(tuple(str(i) for i in range(1,10)))
]

status.head(10)

customer_id customer_name gender_cd gender  birth_day  age postal_cd  \
0   CS021313000114        大野 あや子         1     女性 1981-04-29   37  259-1113   
1   CS037613000071         六角 雅彦         9     不明 1952-04-01   66  136-0076   
3   CS028811000001        堀井 かおり         1     女性 1933-03-27   86  245-0016   
4   CS001215000145         田崎 美紀         1     女性 1995-03-29   24  144-0055   
5   CS020401000016         宮下 達士         0     男性 1974-09-15   44  174-0065   
7   CS029403000008          釈 人志         0     男性 1973-08-17   45  279-0003   
8   CS015804000004         松谷 米蔵         0     男性 1931-05-02   87  136-0073   
9   CS033513000180          安斎 遥         1     女性 1962-07-11   56  241-0823   
10  CS007403000016          依田 満         0     男性 1975-08-18   43  276-0022   
11  CS035614000014        板倉 菜々美         1     女性 1954-07-16   64  154-0015   

                   address application_store_cd  application_date  \
0     神奈川県伊勢原市粟窪**********               S14021          20150905   
1       東京都江東区南砂**********               S13037          20150414   
3   神奈川県横浜市泉区和泉町**********               S14028          20160115   
4      東京都大田区仲六郷**********               S13001          20170605   
5       東京都板橋区若木**********               S13020          20150225   
7       千葉県浦安市海楽**********               S12029          20150515   
8       東京都江東区北砂**********               S13015          20150607   
9   神奈川県横浜市旭区善部町**********               S14033          20150728   
10    千葉県八千代市上高野**********               S12007          20150914   
11    東京都世田谷区桜新町**********               S13035          20150804   

       status_cd  
0   0-00000000-0  
1   0-00000000-0  
3   0-00000000-0  
4   6-20090929-2  
5   0-00000000-0  
7   0-00000000-0  
8   0-00000000-0  
9   6-20080506-5  
10  0-00000000-0  
11  0-00000000-0

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [107]:
status2 = df_customer[df_customer['status_cd'].str.
match(r'^[A-F].*[1-9]$')
]

status2.head(10)

customer_id customer_name gender_cd gender  birth_day  age postal_cd  \
12   CS011215000048         芦田 沙耶         1     女性 1992-02-01   27  223-0062   
68   CS022513000105        島村 貴美子         1     女性 1962-03-12   57  249-0002   
71   CS001515000096         水野 陽子         9     不明 1960-11-29   58  144-0053   
122  CS013615000053         西脇 季衣         1     女性 1953-10-18   65  261-0026   
144  CS020412000161          小宮 薫         1     女性 1974-05-21   44  174-0042   
178  CS001215000097        竹中 あさみ         1     女性 1990-07-25   28  146-0095   
252  CS035212000007        内村 恵梨香         1     女性 1990-12-04   28  152-0023   
259  CS002515000386         野田 コウ         1     女性 1963-05-30   55  185-0013   
293  CS001615000372        稲垣 寿々花         1     女性 1956-10-29   62  144-0035   
297  CS032512000121         松井 知世         1     女性 1962-09-04   56  210-0011   

                      address application_store_cd  application_date  \
12   神奈川県横浜市港北区日吉本町**********               S14011          20150228   
68       神奈川県逗子市山の根**********               S14022          20150320   
71       東京都大田区蒲田本町**********               S13001          20150614   
122    千葉県千葉市美浜区幕張西**********               S12013          20150128   
144       東京都板橋区東坂下**********               S13020          20150822   
178       東京都大田区多摩川**********               S13001          20170315   
252        東京都目黒区八雲**********               S13035          20151013   
259     東京都国分寺市西恋ケ窪**********               S13002          20160410   
293       東京都大田区南蒲田**********               S13001          20170403   
297   神奈川県川崎市川崎区富士見**********               S13032          20150727   

        status_cd  
12   C-20100421-9  
68   A-20091115-7  
71   A-20100724-7  
122  B-20100329-6  
144  B-20081021-3  
178  A-20100211-2  
252  B-20101018-6  
259  C-20100127-8  
293  A-20100104-1  
297  A-20100103-5

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [108]:
pattern = r"\d{3}-\d{3}-\d{4}"
tel = df_store[df_store['tel_no'].str.contains(pattern)]
tel

store_cd store_name prefecture_cd prefecture              address  \
0    S12014       千草台店            12        千葉県      千葉県千葉市稲毛区千草台一丁目   
1    S13002       国分寺店            13        東京都         東京都国分寺市本多二丁目   
2    S14010        菊名店            14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店            14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店            14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店            14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
11   S13052        森野店            13        東京都          東京都町田市森野三丁目   
12   S14028       二ツ橋店            14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店            14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店            14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店            14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店            14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
21   S13016       小金井店            13        東京都         東京都小金井市本町一丁目   
22   S14034      川崎野川店            14       神奈川県         神奈川県川崎市宮前区野川   
26   S14048      中川中央店            14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
27   S12007        佐倉店            12        千葉県            千葉県佐倉市上志津   
28   S14026     辻堂西海岸店            14       神奈川県      神奈川県藤沢市辻堂西海岸二丁目   
29   S13041       八王子店            13        東京都            東京都八王子市大塚   
31   S14049      川崎大師店            14       神奈川県      神奈川県川崎市川崎区中瀬三丁目   
32   S14023        川崎店            14       神奈川県      神奈川県川崎市川崎区本町二丁目   
33   S13018        清瀬店            13        東京都          東京都清瀬市松山一丁目   
35   S14027       南藤沢店            14       神奈川県           神奈川県藤沢市南藤沢   
36   S14021       伊勢原店            14       神奈川県       神奈川県伊勢原市伊勢原四丁目   
37   S14047       相模原店            14       神奈川県       神奈川県相模原市千代田六丁目   
38   S12013       習志野店            12        千葉県         千葉県習志野市芝園一丁目   
40   S14042       新山下店            14       神奈川県      神奈川県横浜市中区新山下二丁目   
42   S12030        八幡店            12        千葉県          千葉県市川市八幡三丁目   
44   S14025        大和店            14       神奈川県           神奈川県大和市下和田   
45   S14045        厚木店            14       神奈川県         神奈川県厚木市中町二丁目   
47   S12029        東野店            12        千葉県          千葉県浦安市東野一丁目   
49   S12053        高洲店            12        千葉県          千葉県浦安市高洲五丁目   
51   S14024        三田店            14       神奈川県      神奈川県川崎市多摩区三田四丁目   
52   S14006       葛が谷店            14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
0           チバケンチバシイナゲクチグサダイイッチョウメ  043-123-4003   140.1180  35.63559   
1             トウキョウトコクブンジシホンダニチョウメ  042-123-4008   139.4802  35.70566   
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
11             トウキョウトマチダシモリノサンチョウメ  042-123-4030   139.4383  35.55293   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
21          トウキョウトコガネイシホンチョウイッチョウメ  042-123-4015   139.5094  35.70018   
22             カナガワケンカワサキシミヤマエクノガワ  044-123-4044   139.5998  35.57693   
26   カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ  045-123-4051   139.5758  35.54912   
27                    チバケンサクラシカミシヅ  043-123-4001   140.1452  35.71872   
28      カナガワケンフジサワシツジドウニシカイガンニチョウメ  046-123-4040   139.4466  35.32464   
29                トウキョウトハチオウジシオオツカ  042-123-4026   139.4235  35.63787   
31       カナガワケンカワサキシカワ

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [109]:

df_customer['birth_day'] = pd.to_datetime(df_customer['birth_day'])
sort_birth = df_customer.sort_values(by='birth_day')
sort_birth.head(10)

customer_id customer_name gender_cd gender  birth_day  age  \
18817  CS003813000014        村山 菜々美         1     女性 1928-11-26   90   
12328  CS026813000004         吉村 朝陽         1     女性 1928-12-14   90   
15682  CS018811000003         熊沢 美里         1     女性 1929-01-07   90   
15302  CS027803000004         内村 拓郎         0     男性 1929-01-12   90   
1681   CS013801000003         天野 拓郎         0     男性 1929-01-15   90   
7511   CS001814000022         鶴田 里穂         1     女性 1929-01-28   90   
2378   CS016815000002         山元 美紀         1     女性 1929-02-22   90   
4680   CS009815000003         中田 里穂         1     女性 1929-04-08   89   
16070  CS005813000015        金谷 恵梨香         1     女性 1929-04-09   89   
6305   CS012813000013         宇野 南朋         1     女性 1929-04-09   89   

      postal_cd                 address application_store_cd  \
18817  182-0007     東京都調布市菊野台**********               S13003   
12328  251-0043   神奈川県藤沢市辻堂元町**********               S14026   
15682  204-0004      東京都清瀬市野塩**********               S13018   
15302  251-0031  神奈川県藤沢市鵠沼藤が谷**********               S14027   
1681   274-0824     千葉県船橋市前原東**********               S12013   
7511   144-0045     東京都大田区南六郷**********               S13001   
2378   184-0005     東京都小金井市桜町**********               S13016   
4680   154-0014     東京都世田谷区新町**********               S13009   
16070  165-0032      東京都中野区鷺宮**********               S13005   
6305   231-0806  神奈川県横浜市中区本牧町**********               S14012   

       application_date     status_cd  
18817          20160214  0-00000000-0  
12328          20150723  0-00000000-0  
15682          20150403  0-00000000-0  
15302          20151227  0-00000000-0  
1681           20160120  0-00000000-0  
7511           20161012  A-20090415-7  
2378           20150629  C-20090923-C  
4680           20150421  D-20091021-E  
16070          20150506  0-00000000-0  
6305           20150712  0-00000000-0

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [110]:

df_customer['birth_day'] = pd.to_datetime(df_customer['birth_day'])
sort_birth = df_customer.sort_values(by='birth_day', ascending=False)
sort_birth.head(10)

customer_id customer_name gender_cd gender  birth_day  age  \
15639  CS035114000004         大村 美里         1     女性 2007-11-25   11   
7468   CS022103000002        福山 はじめ         9     不明 2007-10-02   11   
10745  CS002113000009        柴田 真悠子         1     女性 2007-09-17   11   
19811  CS004115000014         松井 京子         1     女性 2007-08-09   11   
7039   CS002114000010          山内 遥         1     女性 2007-06-03   11   
3670   CS025115000002         小柳 夏希         1     女性 2007-04-18   11   
12493  CS002113000025        広末 まなみ         1     女性 2007-03-30   12   
15977  CS033112000003         長野 美紀         1     女性 2007-03-22   12   
5716   CS007115000006          福岡 瞬         1     女性 2007-03-10   12   
15097  CS014113000008         矢口 莉緒         1     女性 2007-03-05   12   

      postal_cd                  address application_store_cd  \
15639  156-0053       東京都世田谷区桜**********               S13035   
7468   249-0006      神奈川県逗子市逗子**********               S14022   
10745  184-0014    東京都小金井市貫井南町**********               S13002   
19811  165-0031      東京都中野区上鷺宮**********               S13004   
7039   184-0015    東京都小金井市貫井北町**********               S13002   
3670   245-0018  神奈川県横浜市泉区上飯田町**********               S14025   
12493  184-0015    東京都小金井市貫井北町**********               S13002   
15977  245-0051  神奈川県横浜市戸塚区名瀬町**********               S14033   
5716   285-0845      千葉県佐倉市西志津**********               S12007   
15097  260-0041   千葉県千葉市中央区東千葉**********               S12014   

       application_date     status_cd  
15639          20150619  6-20091205-6  
7468           20160909  0-00000000-0  
10745          20160304  0-00000000-0  
19811          20161120  1-20081231-1  
7039           20160920  6-20100510-1  
3670           20160116  D-20100913-D  
12493          20171030  0-00000000-0  
15977          20150606  0-00000000-0  
5716           20151118  F-20101016-F  
15097          20150622  3-20091108-6

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [111]:
df_receipt = df_receipt.sort_values(by = 'amount', ascending = False)
df_receipt['rank'] = df_receipt['amount'].rank(ascending = False, method = 'dense').astype(int)
df_receipt[['customer_id', 'amount', 'rank']].head(10)

customer_id  amount  rank
1202   CS011415000006   10925     1
62317  ZZ000000000000    6800     2
54095  CS028605000002    5780     3
10320  ZZ000000000000    5480     4
4632   CS015515000034    5480     4
72747  ZZ000000000000    5480     4
97294  CS021515000089    5440     5
28304  ZZ000000000000    5440     5
84867  ZZ000000000000    5280     6
79485  ZZ000000000000    5280     6

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [112]:
df_receipt = df_receipt.sort_values(by = 'amount', ascending = False)
df_receipt['rank'] = df_receipt['amount'].rank(ascending = False, method = 'first').astype(int)
df_receipt[['customer_id', 'amount', 'rank']].head(10)

customer_id  amount  rank
1202   CS011415000006   10925     1
62317  ZZ000000000000    6800     2
54095  CS028605000002    5780     3
72747  ZZ000000000000    5480     4
4632   CS015515000034    5480     5
10320  ZZ000000000000    5480     6
28304  ZZ000000000000    5440     7
97294  CS021515000089    5440     8
82209  ZZ000000000000    5280     9
20754  ZZ000000000000    5280    10

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [113]:
num_count = df_receipt.value_counts()
num_count

sales_ymd  sales_epoch  store_cd  receipt_no  receipt_sub_no  customer_id     product_cd  quantity  amount  rank  
20191031   1572480000   S14050    1132        1               ZZ000000000000  P050701009  1         138     65462     1
20170101   1483228800   S12013    1112        1               ZZ000000000000  P090102011  1         180     49913     1
                                  1142        2               ZZ000000000000  P070901024  1         198     43529     1
                                  1172        2               ZZ000000000000  P070202007  1         138     64385     1
                        S12014    1152        1               CS014411000048  P080401010  1         110     74013     1
                                                                                                                     ..
                        S13001    1162        1               CS001414000126  P070705004  1         138     66030     1
                                  1122       

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [114]:
id_count = df_receipt['customer_id'].nunique()
id_count

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
group_sum = df_receipt.groupby('store_cd')[['amount', 'quantity']].sum()
group_sum.head(3)


amount  quantity
store_cd                  
S12007    638761      2099
S12013    787513      2425
S12014    725167      2358

In [ ]:
# %sql
# select
#  store_cd,
# sum(amount) as total_amount,
# sum(quantity) as total_quantity
# from receipt
# group by store_cd

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [116]:
group_id = df_receipt.groupby('customer_id')[['sales_ymd']].max()
group_id.head(10)

sales_ymd
customer_id              
CS001113000004   20190308
CS001114000005   20190731
CS001115000010   20190405
CS001205000004   20190625
CS001205000006   20190224
CS001211000025   20190322
CS001212000027   20170127
CS001212000031   20180906
CS001212000046   20170811
CS001212000070   20191018

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [117]:
group_id = df_receipt.groupby('customer_id')[['sales_ymd']].min()
group_id.head(10)

sales_ymd
customer_id              
CS001113000004   20190308
CS001114000005   20180503
CS001115000010   20171228
CS001205000004   20170914
CS001205000006   20180207
CS001211000025   20190322
CS001212000027   20170127
CS001212000031   20180906
CS001212000046   20170811
CS001212000070   20191018

---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [118]:
group_id_ = df_receipt.groupby('customer_id')['sales_ymd'].agg(['min', 'max'])

group_dif = group_id_[group_id_['min'] != group_id_['max']]
group_dif.head(10)

min       max
customer_id                       
CS001114000005  20180503  20190731
CS001115000010  20171228  20190405
CS001205000004  20170914  20190625
CS001205000006  20180207  20190224
CS001214000009  20170306  20190902
CS001214000017  20180828  20191006
CS001214000048  20171109  20190929
CS001214000052  20180208  20190617
CS001215000005  20170206  20181021
CS001215000040  20170214  20171022

---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [119]:
store_amount_dis = df_receipt.groupby('store_cd')['amount'].mean().sort_values(ascending = False)
store_amount_dis.head(5)

store_cd
S13052    402.867470
S13015    351.111960
S13003    350.915519
S14010    348.791262
S13001    348.470386
Name: amount, dtype: float64

---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [120]:
store_amount_dis = df_receipt.groupby('store_cd')['amount'].median().sort_values(ascending = False)
store_amount_dis.head(5)

store_cd
S13052    190.0
S14010    188.0
S14050    185.0
S13018    180.0
S14040    180.0
Name: amount, dtype: float64

---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [121]:
store_amount_dis = df_receipt.groupby('store_cd')['product_cd'].apply(lambda x: x.mode().tolist())
store_amount_dis.head(10)

store_cd
S12007    [P060303001]
S12013    [P060303001]
S12014    [P060303001]
S12029    [P060303001]
S12030    [P060303001]
S13001    [P060303001]
S13002    [P060303001]
S13003    [P071401001]
S13004    [P060303001]
S13005    [P040503001]
Name: product_cd, dtype: object

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [122]:
store_amount_dis = df_receipt.groupby('store_cd')['amount'].var().sort_values(ascending = False)
store_amount_dis.head(5)

store_cd
S13052    441863.252526
S14011    306442.242432
S14034    297068.392740
S13001    295558.842618
S13015    295427.197086
Name: amount, dtype: float64

---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [123]:
store_amount_dis = df_receipt.groupby('store_cd')['amount'].std().sort_values(ascending = False)
store_amount_dis.head(5)

store_cd
S13052    664.727954
S14011    553.572256
S14034    545.039808
S13001    543.653237
S13015    543.532149
Name: amount, dtype: float64

---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [124]:
amount_4q  = df_receipt['amount'].quantile([0, 0.25, 0.5, 0.75, 1])
amount_4q

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [125]:
amount_ = df_receipt.groupby('store_cd')['amount'].mean()
amount_330 = amount_[amount_ >= 330]
amount_330

store_cd
S12013    330.194130
S13001    348.470386
S13003    350.915519
S13004    330.943949
S13015    351.111960
S13019    330.208616
S13020    337.879932
S13052    402.867470
S14010    348.791262
S14011    335.718333
S14026    332.340588
S14045    330.082073
S14047    330.077073
Name: amount, dtype: float64

---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [126]:
amount_id = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
sum_amount = amount_id.groupby('customer_id')['amount'].mean()
sum_amount 


customer_id
CS001113000004    649.000000
CS001114000005    156.500000
CS001115000010    507.333333
CS001205000004    198.800000
CS001205000006    278.083333
                     ...    
CS051212000001    168.000000
CS051513000004    275.500000
CS051515000002    132.500000
CS052212000002     96.000000
CS052514000001     89.000000
Name: amount, Length: 8306, dtype: float64

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [127]:
amount_id = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
mean_amount = amount_id.groupby('customer_id')['amount'].mean()
all_mean = mean_amount.mean()

mean_amount_1 = mean_amount[mean_amount >= all_mean]
mean_amount_1.head(10)

customer_id
CS001113000004     649.000000
CS001115000010     507.333333
CS001214000009     334.642857
CS001214000017     344.333333
CS001214000052     469.916667
CS001215000040     349.600000
CS001215000097    1144.000000
CS001215000145     437.500000
CS001215000158     337.250000
CS001304000004     624.000000
Name: amount, dtype: float64

---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [128]:
# 内部結合の実行（store_cd をキーに）
df_merged = pd.merge(df_receipt, df_store[['store_cd', 'store_name']], on='store_cd', how='inner')

# 上位10件を表示
print(df_merged.head(10))


   sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20190903   1567468800   S14011        1192               2   
1   20190304   1551657600   S14046        1152               2   
2   20170921   1505952000   S14028        1192               2   
3   20190724   1563926400   S13009        1192               2   
4   20170721   1500595200   S13015        1112               1   
5   20190103   1546473600   S14021        1162               2   
6   20180621   1529539200   S13004        1172               2   
7   20190209   1549670400   S14021        1142               2   
8   20180331   1522454400   S14027        1142               2   
9   20180108   1515369600   S13001        1122               1   

      customer_id  product_cd  quantity  amount  rank store_name  
0  CS011415000006  P080104055         1   10925     1      日吉本町店  
1  ZZ000000000000  P090302118         1    6800     2       北山田店  
2  CS028605000002  P090907210         1    5780     3       二ツ橋店  
3  ZZ

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [129]:
print(df_product.columns)
print(df_category.columns)


Index(['﻿product_cd', 'category_major_cd', 'category_medium_cd',
       'category_small_cd', 'unit_price', 'unit_cost'],
      dtype='object')
Index(['﻿category_major_cd', 'category_major_name', 'category_medium_cd',
       'category_medium_name', 'category_small_cd', 'category_small_name'],
      dtype='object')


In [130]:
df_merged_2 = pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']], on = 'category_small_cd', how = 'inner' )
df_merged_2.head(10)

﻿product_cd category_major_cd category_medium_cd category_small_cd  \
0  P040101001                04               0401            040101   
1  P040101002                04               0401            040101   
2  P040101003                04               0401            040101   
3  P040101004                04               0401            040101   
4  P040101005                04               0401            040101   
5  P040101006                04               0401            040101   
6  P040101007                04               0401            040101   
7  P040101008                04               0401            040101   
8  P040101009                04               0401            040101   
9  P040101010                04               0401            040101   

   unit_price  unit_cost category_small_name  
0       198.0      149.0                 弁当類  
1       218.0      164.0                 弁当類  
2       230.0      173.0                 弁当類  
3       248.0      186.0                 弁当類  
4       268.0      201.0                 弁当類  
5       298.0      224.0                 弁当類  
6       338.0      254.0                 弁当類  
7       420.0      315.0                 弁当類  
8       498.0      374.0                 弁当類  
9       580.0      435.0                 弁当類

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [131]:
common_columns = set(df_customer.columns) & set(df_receipt.columns)
common_columns

{'customer_id'}

In [132]:
df_women_z = df_customer[(df_customer['gender_cd'] == '1') & 
                         (~df_customer['customer_id'].str.startswith('z'))]
df_sales = df_receipt.groupby('customer_id', as_index = False)['amount'].sum()

df_result = pd.merge(df_women_z, df_sales, on = 'customer_id', how = 'inner')

df_result['amount'] = df_result['amount'].fillna(0).astype(int)

df_result[['customer_id', 'amount']].head(10)




customer_id  amount
0  CS031415000172    5088
1  CS001215000145     875
2  CS015414000103    3122
3  CS033513000180     868
4  CS011215000048    3444
5  CS040412000191     210
6  CS029415000023    5167
7  CS009315000023     780
8  CS015315000033     576
9  CS023513000066     771

In [133]:
df_receipt.columns


Index(['sales_ymd', 'sales_epoch', 'store_cd', 'receipt_no', 'receipt_sub_no',
       'customer_id', 'product_cd', 'quantity', 'amount', 'rank'],
      dtype='object')

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [134]:
valid_customer = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

top_day = df_receipt.groupby('customer_id')['sales_ymd'].nunique().reset_index(name='sales_days').sort_values(by='sales_days', ascending=False).head(29)

top_salse = df_receipt.groupby('customer_id')['amount'].sum().reset_index(name= 'sales_sum').sort_values(by = 'sales_sum', ascending=False).head(20)

top_merge = pd.merge(top_day, top_salse, on = 'customer_id', how = 'outer' )

top_merge

customer_id  sales_days   sales_sum
0   CS001605000009         NaN     18925.0
1   CS006515000023         NaN     18372.0
2   CS007514000094         NaN     15735.0
3   CS007515000107        18.0         NaN
4   CS009414000059        17.0     15492.0
5   CS009415000085        17.0         NaN
6   CS010214000002        21.0         NaN
7   CS010214000010        22.0     18585.0
8   CS011414000106         NaN     18338.0
9   CS011415000006         NaN     16094.0
10  CS012414000097        17.0         NaN
11  CS014214000023        19.0         NaN
12  CS014415000077        18.0         NaN
13  CS015415000185        22.0     20153.0
14  CS016415000101         NaN     16348.0
15  CS016415000141        20.0     18372.0
16  CS017415000097        20.0     23086.0
17  CS021514000045        19.0         NaN
18  CS021515000056        18.0         NaN
19  CS021515000089         NaN     17580.0
20  CS021515000172        19.0         NaN
21  CS021515000211        18.0         NaN
22  CS022515000028        18.0         NaN
23  CS022515000150        17.0         NaN
24  CS022515000226        19.0         NaN
25  CS028214000013        17.0         NaN
26  CS028415000007        21.0     19127.0
27  CS028515000100        17.0         NaN
28  CS030214000008        18.0         NaN
29  CS030415000034         NaN     15468.0
30  CS031414000051        19.0     19202.0
31  CS031414000059        17.0         NaN
32  CS031414000073        18.0         NaN
33  CS032414000072         NaN     16563.0
34  CS032415000209        18.0         NaN
35  CS034415000047         NaN     16083.0
36  CS035414000024         NaN     17615.0
37  CS038415000104         NaN     17847.0
38  CS039414000052        19.0         NaN
39  CS040214000008        23.0         NaN
40  ZZ000000000000      1034.0  12395003.0

]---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [135]:

df_cross = pd.merge(df_store, df_product, how='cross')

count = len(df_cross)

count


531590

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [139]:
sales = df_receipt.groupby('sales_ymd')['amount'].sum().sort_index()
sales = sales.reset_index() 
sales['amount_diff'] = sales['amount'].diff()
sales.head(10)



sales_ymd  amount  amount_diff
0   20170101   33723          NaN
1   20170102   24165      -9558.0
2   20170103   27503       3338.0
3   20170104   36165       8662.0
4   20170105   37830       1665.0
5   20170106   32387      -5443.0
6   20170107   23415      -8972.0
7   20170108   24737       1322.0
8   20170109   26718       1981.0
9   20170110   20143      -6575.0

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [141]:
def sales_diff(df_receipt):
    sales = df_receipt.groupby('sales_ymd')['amount'].sum().sort_index().reset_index()
    sales['amount_diff_1'] = sales['amount'].diff(1)
    sales['amount_diff_2'] = sales['amount'].diff(2)
    sales['amount_diff_3'] = sales['amount'].diff(3)

    return sales

sales = sales_diff(df_receipt)
sales.head(10)


sales_ymd  amount  amount_diff_1  amount_diff_2  amount_diff_3
0   20170101   33723            NaN            NaN            NaN
1   20170102   24165        -9558.0            NaN            NaN
2   20170103   27503         3338.0        -6220.0            NaN
3   20170104   36165         8662.0        12000.0         2442.0
4   20170105   37830         1665.0        10327.0        13665.0
5   20170106   32387        -5443.0        -3778.0         4884.0
6   20170107   23415        -8972.0       -14415.0       -12750.0
7   20170108   24737         1322.0        -7650.0       -13093.0
8   20170109   26718         1981.0         3303.0        -5669.0
9   20170110   20143        -6575.0        -4594.0        -3272.0

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
import pandas as pd

# 結合
df_merged = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')

# 年代作成
df_merged['age_group'] = (df_merged['age'] // 10 * 10).astype(str) + '代'

# 性別コードを整数に変換＋マッピング
df_merged['gender_cd'] = pd.to_numeric(df_merged['gender_cd'], errors='coerce')
df_merged['gender_label'] = df_merged['gender_cd'].map({0: '男性', 1: '女性', 9: '不明'})

# クロス集計（年代 × 性別）
summary = pd.pivot_table(
    df_merged,
    values='amount',
    index='age_group',
    columns='gender_label',
    aggfunc='sum',
    fill_value=0
)

# 列順と表示
summary = summary[['女性', '男性', '不明']].reset_index()
print(summary)


gender_label age_group       女性      男性      不明
0                  10代   149836    1591    4317
1                  20代  1363724   72940   44328
2                  30代   693047  177322   50441
3                  40代  9320791   19355  483512
4                  50代  6685192   54320  342923
5                  60代   987741  272469   71418
6                  70代    29764   13435    2427
7                  80代   262923   46360    5111
8                  90代     6260       0       0


In [148]:
print(df_customer['gender_cd'].isnull().sum())


0


---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

# これで１００本終わりです。おつかれさまでした！